In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [108]:
"""
1. Begin/Get data into usable form
"""
#Import everything
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib import pyplot
from sklearn import datasets, preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
import math

#Read in files
excelsheet = pd.read_csv('/content/drive/Shared drives/DATATHON 2020/training2.csv')
df = pd.DataFrame(excelsheet) # get original dataframe
#excelsheettest = pd.read_csv('/content/drive/Shared drives/DATATHON 2020/testing.csv')
#dftest = pd.DataFrame(excelsheettest) # get testing dataframe
m = list(df.columns) # list of column names

#Do not consider columns for which the bit model ID is 17
#df = df[df.bit_model_id != 17]

# take out everything with rate of penetration extremely high
df = df[df.rate_of_penetration < 300]

# centerpoint
df['sample_thickness'] = df['max_depth'] - df['min_depth']
df['sample_depth'] = df['min_depth'] - .5*df['sample_thickness']



#df1 = df[['bit_model_id', 'drillbit_size', 'sample_depth', 'surface_weight_on_bit', 'surface_rpm']]


#df1 is the dataframe with no IDs and rate of penetration removed, also min_depth
df1 = df[['wellbore_chev_no_id', 'area_id', 'formation_id', 'bit_model_id', 'drillbit_size', 'sample_depth', 'surface_weight_on_bit', 'surface_rpm']]
#df1test = dftest[['wellbore_chev_no_id', 'area_id', 'formation_id', 'bit_model_id', 'drillbit_size', 'sample_depth', 'surface_weight_on_bit', 'surface_rpm']]


k = list(df1.columns) # list of column names
print("hellp", k)

#Take out the "y" column (rate of penetration)
dfy = df[['rate_of_penetration']].copy()

"""
2. Preprocessing/Feature engineering
"""
#Singular Value Decompostion
#Principal Components Analysis

"""
3. Test the model
"""
#Split the training data
X = df1.values # X values in matrix
y = dfy.values # Y values in matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

poly_reg = PolynomialFeatures(degree=4) # fit a polynomial with degree 4
X_poly = poly_reg.fit_transform(X_train)
pol_reg = LinearRegression()
pol_reg.fit(X_poly, y_train)

#print("xtestlength", X_test[0])

y_pred = []
for u in range(0,len(X_test)):
  yay = pol_reg.predict(poly_reg.fit_transform([X_test[u]]))
  #print('yayitself',yay[0][0])
  #print('typeyay',type(yay[0][0]))
  y_pred.append(yay[0][0])


print('y_pred',y_pred)
#yay = pol_reg.predict(poly_reg.fit_transform([X_test[5]]))
#ytest5 = y_test[5]


#Evaluate its performance

errs = []
rmse = 0
for v in range(0,len(y_test)):

  errs.append((abs(y_test[v] - y_pred[v])/(y_test[v]))*100)
  rmse = rmse + math.pow((y_test[v] - y_pred[v]),2)

mean2 = np.mean(errs)
rmse = rmse/len(y_test)
rmse = math.sqrt(rmse)


finaldf = df[['segment_id']].copy()
#finaldf['rate_of_penetration'] = y_pred 

print(finaldf)

print('Mean percentage error:', mean2)
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', rmse)

'''
4. Run the model on the test data

#Run the model
X = df1.values
y = dfy.values
X_test = df1test.values

#Create and run the model
regressor = LinearRegression()  
regressor.fit(X, y)
y_pred = regressor.predict(X_test)

#Export the results
pd.DataFrame(y_pred).to_csv("/content/drive/Shared drives/DATATHON 2020/finalresult.csv")
'''

hellp ['wellbore_chev_no_id', 'area_id', 'formation_id', 'bit_model_id', 'drillbit_size', 'sample_depth', 'surface_weight_on_bit', 'surface_rpm']
y_pred [134.65062516208494, 220.72245727288737, 109.67253664931073, 83.48269009811395, 75.13954565568295, 117.1242836520425, 137.62312141840692, 163.33553221015086, 152.97791720179066, 178.6474066731415, 86.88128123976526, 127.83420742755408, 95.45758315520504, 16.68256403740071, 55.89678070896775, 133.96780596444526, 176.83842456092415, 64.25368894895064, 135.5667706479768, 149.8548793560331, 166.71656267016107, 47.439869565177446, 45.45881642350582, 68.111268072097, 52.46248742956942, 126.34925096328413, 137.93657211831078, 76.02658028207648, 217.35981421507825, 118.7743535088259, 55.18623012934262, 108.7471160869739, 75.2791007314687, 87.44216817929646, 41.801670480348434, 91.43080328016958, 82.89777329324437, 135.58249970855115, 62.380034117077344, 150.35303222341787, 97.24487527666892, 59.25751593137386, 77.19300108952731, 70.35176822880

'\n4. Run the model on the test data\n\n#Run the model\nX = df1.values\ny = dfy.values\nX_test = df1test.values\n\n#Create and run the model\nregressor = LinearRegression()  \nregressor.fit(X, y)\ny_pred = regressor.predict(X_test)\n\n#Export the results\npd.DataFrame(y_pred).to_csv("/content/drive/Shared drives/DATATHON 2020/finalresult.csv")\n'

In [107]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib import pyplot
from sklearn import datasets, preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import math


pd.set_option('display.max_columns', None)
excelsheet = pd.read_csv('/content/drive/Shared drives/DATATHON 2020/mytestscoring.csv')
df = pd.DataFrame(excelsheet) # get original dataframe


def clean(df):
  # remove non numeric values for various columns
  for v in range(0,df.shape[0]):
    #wellbore_chev_no_id
    df.iloc[v,1] = int(df.iloc[v,1][17:])
    #area_id
    df.iloc[v,2] = int(df.iloc[v,2][5:])
    #formation_id
    df.iloc[v,3] = int(df.iloc[v,3][10:])
    #bit_model_id
    df.iloc[v,4] = int(df.iloc[v,4][10:])

      # centerpoint
  df['sample_thickness'] = df['max_depth'] - df['min_depth']
  df['sample_depth'] = df['min_depth'] - .5*df['sample_thickness']

  newdf = df[['wellbore_chev_no_id', 'area_id', 'formation_id', 'bit_model_id', 'drillbit_size', 'sample_depth', 'surface_weight_on_bit', 'surface_rpm']]
  return(newdf)


outputlabeltest = clean(df)
print(outputlabeltest)

#pd.DataFrame(outputlabeltest).to_csv("/content/drive/Shared drives/DATATHON 2020/outputlabeltest2.csv")


      wellbore_chev_no_id  area_id  formation_id  bit_model_id  drillbit_size  \
0                       0        0            62             1          12.25   
1                       0        0            61             1          12.25   
2                       0        0            59             1          12.25   
3                       0        0            65             1          12.25   
4                       0        0            65             1          12.25   
...                   ...      ...           ...           ...            ...   
6833                    9        0            88             4          12.25   
6834                    9        0            87             4          12.25   
6835                    9        0            87             4          12.25   
6836                    9        0            87             4          12.25   
6837                    9        0            87             4          12.25   

      sample_depth  surface

In [114]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib import pyplot
from sklearn import datasets, preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn import metrics
import math


excelsheetz = pd.read_csv('/content/drive/Shared drives/DATATHON 2020/scoring.csv')
dfz = pd.DataFrame(excelsheetz) # get original dataframe

# function to clean test data csv, return clean pandas dataframe
def clean(df):
  # remove non numeric values for various columns
  for v in range(0,df.shape[0]):
    #wellbore_chev_no_id
    df.iloc[v,1] = int(df.iloc[v,1][17:])
    #area_id
    df.iloc[v,2] = int(df.iloc[v,2][5:])
    #formation_id
    df.iloc[v,3] = int(df.iloc[v,3][10:])
    #bit_model_id
    df.iloc[v,4] = int(df.iloc[v,4][10:])
  
  # centerpoint
  df['sample_thickness'] = df['max_depth'] - df['min_depth']
  df['sample_depth'] = df['min_depth'] - .5*df['sample_thickness']
  
  newdf = df[['wellbore_chev_no_id', 'area_id', 'formation_id', 'bit_model_id', 'drillbit_size', 'sample_depth', 'surface_weight_on_bit', 'surface_rpm']]

  return(newdf)

dftest = clean(dfz)


#Read in training data files
excelsheet = pd.read_csv('/content/drive/Shared drives/DATATHON 2020/training2.csv')
df = pd.DataFrame(excelsheet) # get original dataframe
m = list(df.columns) # list of column names

# take out everything with rate of penetration extremely high
df = df[df.rate_of_penetration < 300]

# centerpoint
df['sample_thickness'] = df['max_depth'] - df['min_depth']
df['sample_depth'] = df['min_depth'] - .5*df['sample_thickness']


#df1 is the TRAINING dataframe with no IDs and rate of penetration removed, sample_depth replacing min and max depth
df1 = df[['wellbore_chev_no_id', 'area_id', 'formation_id', 'bit_model_id', 'drillbit_size', 'sample_depth', 'surface_weight_on_bit', 'surface_rpm']]
k = list(df1.columns) # list of column names

#Take out the "y" column (rate of penetration)
dfy = df[['rate_of_penetration']].copy()

# VALUES FOR TRAINING
X_train = df1.values # X values in matrix
y_train = dfy.values # Y values in matrix

# VALUES FOR TESTING
X_test = dftest.values

print('xlen',len(X_test))

poly_reg = PolynomialFeatures(degree=4) # fit a polynomial with degree 4
X_poly = poly_reg.fit_transform(X_train)
pol_reg = LinearRegression()
pol_reg.fit(X_poly, y_train)


y_pred = []
for u in range(0,len(X_test)):
  yay = pol_reg.predict(poly_reg.fit_transform([X_test[u]]))
  #print('yayitself',yay[0][0])
  #print('typeyay',type(yay[0][0]))
  y_pred.append(yay[0][0])


print('y_pred',y_pred)
print('lenypred',len(y_pred))



finaldf = dfz[['segment_id']].copy()
finaldf['rate_of_penetration'] = y_pred 


pd.DataFrame(finaldf).to_csv("/content/drive/Shared drives/DATATHON 2020/finalsubmission.csv", index = False)
print(finaldf)



xlen 1567
y_pred [229.37751989236472, 142.2028139208739, 140.117008168461, 142.41579003255632, 127.29453674013476, 118.67342361609997, 120.8144782526445, 113.30258665656771, 114.13657434643716, 115.15071868267486, 125.40638125968115, 136.34815635478492, 142.15292483116005, 142.16256494121336, 141.64233851171772, 140.29855584633722, 162.06423670949223, 155.43775892503766, 155.0775855476169, 147.9789986585094, 181.9380978509147, 181.25538358657698, 187.45122242674265, 186.69781953124212, 180.90653731334794, 186.70569677072135, 182.1323044297687, 179.2691143753906, 163.6711355119096, 174.77282388360797, 177.58363589724812, 187.36631695289765, 188.7972758829292, 177.97576400774255, 177.22319222389956, 181.1860987881447, 188.76888350136065, 177.6460453734653, 179.87499976454336, 181.0874306409088, 183.85363372340586, 180.913255896444, 176.42721892851426, 172.9736503665738, 174.7275074503491, 173.46718771411133, 188.73003209214912, 199.5005475295366, 198.89827486472961, 188.3296976047231, 18